# Generative Adversarial Network on MNIST 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
torch.manual_seed(7)
np.random.seed(7)

In [9]:
BATCH_SIZE = 64 
G_LR = 0.01 # Learning rate for G network
D_LR = 0.01 # Learning rate for D network
INPUT_DIMENSION = 784
INPUT_NUMBERS = 100
N_NODES = 128

## 1. Load MNIST Data

In [3]:
mnist_trainset=torchvision.datasets.MNIST('MNIST',download=True)

## 2. Create Two Networks

In [ ]:
Generator = nn.Sequential(
    nn.Linear(INPUT_NUNBERS,N_NODES)
    nn.ReLU()
    nn.Linear(N_NODES,INPUT_DIMENSION)
)

Discriminator = nn.Sequential(
    nn.Linear(INPUT_DIMENSION,N_NODES)
    nn.ReLU()
    nn.Linear(N_NODES,1)
    nn.Sigmoid()
)

## 3. Model Training